## Preliminary Operations


In [ ]:
import requests

In [1]:
# 1) go to https://developer.spotify.com/console/get-audio-analysis-track/?id=06AKEBrKUckW0KREUWRnvT
# 2) press "try it"
# 3) login
# 4) agree 
# 5) execute this cell and give the script the token (see above)
if "token" not in locals(): # if you have not inserted the token 
    token=input("Give me the token")

header={"Authorization": "Bearer %s"%token}


In [ ]:
# %% Search api: first info

params={"q": "Staying alive bee gees", "type": "track"}
search_url="https://api.spotify.com/v1/search"
req=requests.get(url=search_url, params=params,headers=header)
assert req.status_code==200, req.content
answer=req.json()
items=answer["tracks"]["items"]
first_result=items[0]
print("First result")
print("Author: %s"%first_result["artists"][0]["name"])
print("Name: %s"%first_result["name"])
print("Preview url: %s"%first_result["preview_url"])
print("Id on spotify: %s"%first_result["id"])

In [ ]:
# %% Audio feature APIs

modes=["minor", "major"]
key_tonal=["C","C#", "D","D#","E","F","F#","G","G#","A","A#","B"]

audio_feature_url="https://api.spotify.com/v1/audio-features"
params={"ids":first_result["id"]}
req=requests.get(url=audio_feature_url, params=params, headers=header)
audio_features=req.json()["audio_features"][0]
#print(audio_features)
print("Duration: %.3f seconds"%(audio_features["duration_ms"]/1000))
print("BPM: %d"%audio_features["tempo"])
print("Key: %s-%s"%(key_tonal[audio_features["key"]], 
                   modes[audio_features["mode"]]))

for feature in ["danceability", "energy", "speechiness", "acousticness","liveness","instrumentalness","valence"]:
    print("The %s of the song is %1.f %%"%(feature, 100*audio_features[feature]))


## Playlist


In [ ]:
import os
import json
import time
import requests
import urllib
import numpy as np
os.chdir(os.path.abspath(os.path.dirname(__file__)))

audio_feature_url="https://api.spotify.com/v1/audio-features"
add_item_playlist_url="https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
create_playlist_url="https://api.spotify.com/v1/users/{user_id}/playlists"

In [ ]:
# %% Get the token
# 1) go to https://developer.spotify.com/console/post-playlists/
# 2) press "get token"
# 3) remember to include playlist-modify-public 
# 4) login
# 5) agree 
# 6) execute this cell and give the script the token (see above)
if "token" not in locals(): # if you have not inserted the token 
    token=input("Please, give me your token\n")
header={"Authorization": "Bearer %s"%token}


In [ ]:
# %% Get the list of songs

assert os.path.exists("list_of_songs.json"), "Please put here a list of songs"
with open("list_of_songs.json",'r') as fp:
    ids=json.load(fp)["ids"]

In [ ]:
# %% Get the audio features
audio_features=[]
for id_ in ids:
    params={"ids":id_}
    req=requests.get(url=audio_feature_url, params=params, headers=header)
    assert req.status_code==200, req.content
    audio_features_song=req.json()["audio_features"][0]
    audio_features.append(audio_features_song)
    time.sleep(1) # wait 1 second between the questions

In [ ]:
def sort_songs(audio_features):
    """"Receive audio features and sort them according to your criterion"

    Args:
        audio_features (list of dictionaries): List of songs with audio features

    Returns:
        list of dict: the sorted list
    """
    sorted_songs=[]

    # Random shuffle: replace it!
    if TEACHER_CODE:
        random_idxs=np.random.permutation(len(audio_features))
        for idx in random_idxs:
            sorted_songs.append(audio_features[idx])
    else:
        # your code here
        denceability = np.array([af["danceability"] for af in audio_features])
        idx = np.argsort(denceability)
        N_third = int(len(audio_features)/3)
        low_dance_idx = idx[0:N_third]
        mid_dance_idx = idx[N_third: 2*N_third]
        high_dance_idx = idx[N_third*2:]
        sorted_idx = np.stack([mid_dance_idx, high_dance_idx, low_dance_idx[::-1]])
        for idxs in sorted_idx:
            sorted_songs.append(audio_features[idxs])
        
    return sorted_songs

In [ ]:
# %% Now let's create some way to organize them!

shuffled_songs=sort_songs(audio_features)

In [ ]:
# %% Create the playlist
# Go to https://open.spotify.com/ , top right corner, press "Account"
# look at your username or user_id
name_playlist=input("What's the name of the playlist you want to create?\n")
user_id=input("What's your username?\n")

params={"name":name_playlist, "description": "made during cpac!"}

In [ ]:
req=requests.post(url=create_playlist_url.format(user_id=user_id), 
                    json=params, headers=header)
assert req.status_code==201, req.content
playlist_info=req.json()
print("Playlist created with url %s"%playlist_info["external_urls"]["spotify"])

In [ ]:
# %% Populating the playlist
# Doc at https://developer.spotify.com/documentation/web-api/reference/playlists/add-tracks-to-playlist/
uris=[]
for song in shuffled_songs:
    uris.append(song["uri"])
params={"uris":uris, }
req=requests.post(url=add_item_playlist_url.format(playlist_id=playlist_info["id"]), json=params, headers=header)
assert req.status_code==201, req.content
playlist_info_songs=req.json()